In [1]:
import time
from urllib.request import urlopen # url 웹페이지 오픈
from bs4 import BeautifulSoup

<br>

#### 1. 메일 발송을 위한 세팅 및 함수 선언

In [2]:
# 참고하여 지메일 사전 세팅 필요 @ https://goo.gl/ET1RMc

import smtplib
from email.mime.text import MIMEText

def sendMail(sender, receiver, msg):
    smtp = smtplib.SMTP_SSL('smtp.gmail.com', 465)
    smtp.login(sender, '?????') # 지메일 "앱 비밀번호" 를 입력해주세요.
    msg = MIMEText(msg)
    msg['Subject'] = 'Product is available!'
    smtp.sendmail(sender, receiver, msg.as_string())
    smtp.quit()

<br>

#### 2. 와디즈 프로젝트 웹페이지 소스코드 로딩

In [3]:
url = 'https://www.wadiz.kr/web/campaign/detail/41278'

In [5]:
webpage = urlopen(url)
source = BeautifulSoup(webpage, 'html.parser')

target = source.find_all('button', {'class':'rightinfo-reward-list'})

<br>

#### 3-1. button 태그로부터 특정 펀딩 가격 텍스트 뽑아내기

In [9]:
target[0]

<button class="rightinfo-reward-list ing" onclick="backMoney('103219', 'false')">
<div class="top-info">
<dl class="reward-info">
<dt>
                        41,000원 펀딩
                      </dt>
<dd>
<p class="reward-name">스페셜 얼리버드 [싱글]</p>
<p>구성제품 룸바냅 단품 1개<br/>[스페셜 얼리버드 51% 혜택]<br/><br/>*오로라 퍼플은 9월 말 배송 예정입니다</p>
</dd>
</dl>
<ul class="data-info">
<li class="shipping" style="width: 100%; margin-bottom: 5px;">배송비<em>2,500원</em></li>
<li class="date" style="width: 100%;">리워드 발송 시작일
                      
                      
                      
                        
                        
                        <em>2019년 10월 말 (21~말일) 예정</em>
</li>
</ul>
<p class="reward-qty">제한수량 <strong>1400</strong>개   <em>현재 934개 남음!</em></p>
<p class="reward-soldcount">총 <strong>466</strong>개 펀딩완료</p>
</div>
<p class="hover-text">이 리워드 펀딩하기</p>
</button>

In [11]:
# <button>.find('dt').text.strip() == '41,000원 펀딩':
target[0].find('dt').text.strip()

'41,000원 펀딩'

<br>

#### 3-2. button 태그로부터 class attribute 값 뽑아내기

In [10]:
target[0].attrs

{'class': ['rightinfo-reward-list', 'ing'],
 'onclick': "backMoney('103219', 'false')"}

In [12]:
# len(<button>.attrs['class']) == 2:
len(target[0].attrs['class'])

2


#### 4. 실제 크롤링 코드 구현 
<br>

- button 태그 내부의 펀딩 가격 텍스트를 기준으로 **특정한 가격을 가진 button 태그를 찾고**,
- 해당 **button 태그의 class attribute의 value가 특정한 갯수가 되었을 시**,
- 미리 선언해 둔 **메일 발송 함수를 실행** 
- 위 조건이 만족되기 전에는 check_status의 값이 1로 유지되어 계속해서 while 문을 돌게 됨

In [ ]:
check_status = 1

while check_status == 1:
    webpage = urlopen(url)
    source = BeautifulSoup(webpage, 'html.parser')
    target = source.findAll('button', {'class':'rightinfo-reward-list'})
    
    for item in target: # 각각의 버튼마다 
        if item.find('dt').text.strip() == '99,800원 펀딩': # 금액 텍스트를 체크해서 '99,800원 펀딩'이라고 쓰여져 있을 경우
            if len(item.attrs['class']) == 2: # 해당 태그에 매겨져 있는 class의 갯수가 2개일 경우,
                sendMail('repositivator@gmail.com', 'repositivator@gmail.com', '99800 Available \n https://goo.gl/wG8v1F')
                check_status = 0
    time.sleep(10)

In [ ]:
# sendMail('repositivator@gmail.com', 'repositivator@gmail.com', 'Crawler stopped!')